# Recommender Systems and Choice: A Reproducibility Study

*by Adeline Liem*


## Kilands Data: Low Attractiveness (LA)

### Data Wrangling and Cleaning

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import SnowballStemmer
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import os
from pymer4.models import Lmer
import pymc as pm

In [2]:
# Read datasets

kilands_df = pd.read_csv('kilands_20201103.csv')
cyberfoto_df = pd.read_csv('cyberfoto_20210121.csv')

In [3]:
kilands_df.shape

(14832, 26)

In [4]:
cyberfoto_df.shape

(37005, 24)

In [5]:
kilands_df.describe()

,number of add-to-carts,number of clicks,first click rank,first purchase rank,order value,number of purchases,number of searches in current session,search session length (seconds),number of products displayed to user,number of sessions,whole session length,time to add-to-cart (seconds),time to first click (seconds),time to purchase
count,14832.000000,14832.000000,7422.000000,249.000000,249.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,657.000000,7422.000000,249.000000
mean,0.050499,0.982470,15.801536,12.887550,1638.630602,0.018406,2.158306,151.272923,42.654396,4.500202,514.805218,266.042618,36.330369,471.072289
std,0.258518,1.763807,40.195333,34.877369,2144.580567,0.146424,1.798311,1828.612696,66.212976,13.663486,6762.696042,289.368074,79.857679,360.449835
min,0.000000,0.000000,0.000000,0.000000,39.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,3.000000,1.000000,40.000000
25%,0.000000,0.000000,1.000000,0.000000,409.500000,0.000000,1.000000,0.000000,5.000000,1.000000,18.000000,71.000000,8.000000,195.000000
50%,0.000000,1.000000,4.000000,1.000000,985.000000,0.000000,2.000000,20.000000,24.000000,1.000000,96.000000,157.000000,17.000000,369.000000
75%,0.000000,1.000000,15.000000,7.000000,2090.000000,0.000000,3.000000,92.000000,48.000000,2.000000,344.000000,338.000000,36.000000,655.000000
max,6.000000,50.000000,1634.000000,321.000000,16490.000000,3.000000,14.000000,155064.000000,1416.000000,97.000000,336633.000000,1747.000000,1507.000000,1777.000000


In [6]:
### Removing outliers ###

# Defining bounds for exclusion: search session length

upper_bound_length_kilands = kilands_df['search session length (seconds)'].mean() + \
                             2.5 * kilands_df['search session length (seconds)'].std()
print(upper_bound_length_kilands)

# Count the number of observations with search session length exceeding the upper bound
num_outliers_search = len(kilands_df[kilands_df['search session length (seconds)'] > upper_bound_length_kilands])
print(num_outliers_search)

# Creating new dataset with search session outliers removed
kilands_no_out = kilands_df[kilands_df['search session length (seconds)'] <= upper_bound_length_kilands]

## Same bounds for exclusion and number of outliers in Python and R code

4722.8046632183505
27


In [7]:
# Define bounds for exclusion: clicks
upper_bound_clicks_kilands = kilands_no_out['number of clicks'].mean() + \
                             2.5 * kilands_no_out['number of clicks'].std()
print(upper_bound_clicks_kilands)

# Count the number of observations with clicks exceeding the upper bound
num_outliers_clicks = len(kilands_no_out[kilands_no_out['number of clicks'] > upper_bound_clicks_kilands])
print(num_outliers_clicks)

## Same bounds for exclusion and number of outliers in Python and R code

5.389793656358605
374


In [8]:
kilands_no_out.shape

(14805, 26)

In [9]:
# Creating new dataset with clicks outliers removed
kilands_no_out1 = kilands_no_out[kilands_no_out['number of clicks'] <= upper_bound_clicks_kilands]

In [10]:
kilands_no_out1.shape

(14431, 26)

In [11]:
# Recode High Attractiveness (HA) and Low Attractiveness (LA)
kilands_no_out1.loc[kilands_no_out1['segment'] == "202004_gandalf_rel", 'segment'] = "HA"
kilands_no_out1.loc[kilands_no_out1['segment'] == "202004_kilands_shuffle7", 'segment'] = "LA"

# Keep only certain cols: search ID (1), user (2), segment (4), platform (7), 15, 10, 11, 12, 13,
##, 14, 17, 20, 21, 22, 23, 25, 26, 18
columns_to_keep = [
    kilands_no_out.columns[i - 1] for i in 
    [1, 2, 4, 7, 15, 10, 11, 17, 12, 13, 14, 20, 21, 23, 22, 25, 26, 18]
]
kilands_no_out1 = kilands_no_out[columns_to_keep]

In [12]:
kilands_no_out1.shape

(14805, 18)

In [13]:
# Recode attention_click
kilands_no_out1['attention_click'] = kilands_no_out1['first click rank'].apply(
    lambda x: "Top" if x <= 6 else ("Bottom" if x > 6 else x))


/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/3420092006.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilands_no_out1['attention_click'] = kilands_no_out1['first click rank'].apply(


In [14]:
kilands_no_out1.shape

(14805, 19)

In [15]:
# Recode attention_purchase
kilands_no_out1['attention_purchase'] = kilands_no_out1['first purchase rank'].apply(
    lambda x: "Top" if x <= 6 else ("Bottom" if x > 6 else x))

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/4062554595.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilands_no_out1['attention_purchase'] = kilands_no_out1['first purchase rank'].apply(


In [16]:
kilands_no_out1

,search id,user,segment,platform,number of purchases,number of add-to-carts,number of clicks,search session length (seconds),first click rank,first purchase rank,order value,whole session length,time to add-to-cart (seconds),time to purchase,time to first click (seconds),click positions,purchase positions,number of products displayed to user,attention_click,attention_purchase
0,u:004f5XJdPvnue61i/s:IFEonOw2XywDPmFc/q:q=sisa...,004f5XJdPvnue61i,202004_kilands_shuffle7,mobile,0,0,1,32,23.0,NaN,NaN,32,NaN,NaN,32.0,23,NaN,24,Bottom,NaN
1,u:00U825g1ZwBVX8UW/s:tQCDykE6URsuP1t7/q:q=bilm...,00U825g1ZwBVX8UW,202004_kilands_shuffle7,mobile,0,0,2,73,0.0,NaN,NaN,73,NaN,NaN,13.0,0;0,NaN,8,Top,NaN
2,u:01U04AG9RtpnKveG/s:r0I5bjEYaljUF8Ff/q:q=tras...,01U04AG9RtpnKveG,202004_kilands_shuffle7,desktop,0,0,1,17,0.0,NaN,NaN,17,NaN,NaN,17.0,0,NaN,1,Top,NaN
3,u:01WAyTwhvMkQVpSv/s:HIrTC5hOFuzum34m/q:q=matt...,01WAyTwhvMkQVpSv,202004_gandalf_rel,mobile,0,0,1,12,8.0,NaN,NaN,12,NaN,NaN,12.0,8,NaN,24,Bottom,NaN
4,u:022YZKPnqb7KERVw/s:53Z43z9957vgQKIY/q:q=plas...,022YZKPnqb7KERVw,202004_gandalf_rel,mobile,0,0,1,39,53.0,NaN,NaN,39,NaN,NaN,39.0,53,NaN,72,Bottom,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14827,u:zxw5bXtZLcAzBAPs/s:SVKHkRkMD5Z5dKMh/q:q=rund...,zxw5bXtZLcAzBAPs,202004_gandalf_rel,tablet,0,0,0,61,NaN,NaN,NaN,71,NaN,NaN,NaN,NaN,NaN,120,NaN,NaN
14828,u:zxw5bXtZLcAzBAPs/s:SVKHkRkMD5Z5dKMh/q:q=matt...,zxw5bXtZLcAzBAPs,202004_gandalf_rel,tablet,0,0,0,4,NaN,NaN,NaN,71,NaN,NaN,NaN,NaN,NaN,48,NaN,NaN
14829,u:zyfLPL38d2wolWFK/s:iKyf7aO0Ta2asuqP/q:q=274 ...,zyfLPL38d2wolWFK,202004_gandalf_rel,desktop,0,0,1,33,0.0,NaN,NaN,33,NaN,NaN,33.0,0,NaN,1,Top,NaN
14830,u:zz4XT5jMM6FMGos0/s:ClDePgytzZ4L4MGK/q:q=rio/...,zz4XT5jMM6FMGos0,202004_kilands_shuffle7,tablet,0,1,1,18,2.0,NaN,NaN,33,5.0,NaN,5.0,2,NaN,4,Top,NaN


In [17]:
# Replace semicolons in 'click positions' and 'purchase positions' with ' '
kilands_no_out1['clicks_position'] = kilands_no_out1['click positions'].str.replace(";", " ")
kilands_no_out1['purchase_position'] = kilands_no_out1['purchase positions'].str.replace(";", " ")

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/3077631480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilands_no_out1['clicks_position'] = kilands_no_out1['click positions'].str.replace(";", " ")
/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/3077631480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilands_no_out1['purchase_position'] = kilands_no_out1['purchase positions'].str.replace(";", " ")


In [18]:
kilands_no_out1.shape

(14805, 22)

In [19]:
# Recode top_clicks and bottom_clicks
kilands_no_out1['top_clicks'] = kilands_no_out1['clicks_position'].apply(
    lambda x: sum(int(pos) <= 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
)
kilands_no_out1['bottom_clicks'] = kilands_no_out1['clicks_position'].apply(
    lambda x: sum(int(pos) > 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
)

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/1140803558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilands_no_out1['top_clicks'] = kilands_no_out1['clicks_position'].apply(
/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/1140803558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilands_no_out1['bottom_clicks'] = kilands_no_out1['clicks_position'].apply(


In [20]:
kilands_no_out1.shape

(14805, 24)

In [21]:
# Recode top_purch and bottom_purch
kilands_no_out1['top_purch'] = kilands_no_out1['purchase_position'].apply(
    lambda x: sum(int(pos) <= 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
)

kilands_no_out1['bottom_purch'] = kilands_no_out1['purchase_position'].apply(
    lambda x: sum(int(pos) > 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
)

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/2901962938.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilands_no_out1['top_purch'] = kilands_no_out1['purchase_position'].apply(


In [22]:
kilands_no_out1.shape

(14805, 26)

In [23]:
# Convert `attention_click` and `attention_purchase` to category
kilands_no_out1['attention_click'] = kilands_no_out1['attention_click'].astype('category')
print(kilands_no_out1['attention_click'].describe())
print("\n")

kilands_no_out1['attention_purchase'] = kilands_no_out1['attention_purchase'].astype('category')
print(kilands_no_out1['attention_purchase'].describe())
print("\n")

# Value counts for attention_click
print(kilands_no_out1['attention_click'].value_counts())

count     7411
unique       2
top        Top
freq      4503
Name: attention_click, dtype: object


count     249
unique      2
top       Top
freq      184
Name: attention_purchase, dtype: object


attention_click
Top       4503
Bottom    2908
Name: count, dtype: int64


In [24]:
kilands_no_out1

,search id,user,segment,platform,number of purchases,number of add-to-carts,number of clicks,search session length (seconds),first click rank,first purchase rank,...,purchase positions,number of products displayed to user,attention_click,attention_purchase,clicks_position,purchase_position,top_clicks,bottom_clicks,top_purch,bottom_purch
0,u:004f5XJdPvnue61i/s:IFEonOw2XywDPmFc/q:q=sisa...,004f5XJdPvnue61i,202004_kilands_shuffle7,mobile,0,0,1,32,23.0,NaN,...,NaN,24,Bottom,NaN,23,NaN,0,1,0,0
1,u:00U825g1ZwBVX8UW/s:tQCDykE6URsuP1t7/q:q=bilm...,00U825g1ZwBVX8UW,202004_kilands_shuffle7,mobile,0,0,2,73,0.0,NaN,...,NaN,8,Top,NaN,0 0,NaN,2,0,0,0
2,u:01U04AG9RtpnKveG/s:r0I5bjEYaljUF8Ff/q:q=tras...,01U04AG9RtpnKveG,202004_kilands_shuffle7,desktop,0,0,1,17,0.0,NaN,...,NaN,1,Top,NaN,0,NaN,1,0,0,0
3,u:01WAyTwhvMkQVpSv/s:HIrTC5hOFuzum34m/q:q=matt...,01WAyTwhvMkQVpSv,202004_gandalf_rel,mobile,0,0,1,12,8.0,NaN,...,NaN,24,Bottom,NaN,8,NaN,0,1,0,0
4,u:022YZKPnqb7KERVw/s:53Z43z9957vgQKIY/q:q=plas...,022YZKPnqb7KERVw,202004_gandalf_rel,mobile,0,0,1,39,53.0,NaN,...,NaN,72,Bottom,NaN,53,NaN,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14827,u:zxw5bXtZLcAzBAPs/s:SVKHkRkMD5Z5dKMh/q:q=rund...,zxw5bXtZLcAzBAPs,202004_gandalf_rel,tablet,0,0,0,61,NaN,NaN,...,NaN,120,NaN,NaN,NaN,NaN,0,0,0,0
14828,u:zxw5bXtZLcAzBAPs/s:SVKHkRkMD5Z5dKMh/q:q=matt...,zxw5bXtZLcAzBAPs,202004_gandalf_rel,tablet,0,0,0,4,NaN,NaN,...,NaN,48,NaN,NaN,NaN,NaN,0,0,0,0
14829,u:zyfLPL38d2wolWFK/s:iKyf7aO0Ta2asuqP/q:q=274 ...,zyfLPL38d2wolWFK,202004_gandalf_rel,desktop,0,0,1,33,0.0,NaN,...,NaN,1,Top,NaN,0,NaN,1,0,0,0
14830,u:zz4XT5jMM6FMGos0/s:ClDePgytzZ4L4MGK/q:q=rio/...,zz4XT5jMM6FMGos0,202004_kilands_shuffle7,tablet,0,1,1,18,2.0,NaN,...,NaN,4,Top,NaN,2,NaN,1,0,0,0


In [25]:
# Create dummy binary variables for add-to-carts and purchases
kilands_no_out1['carts'] = kilands_no_out1['number of add-to-carts'].apply(
    lambda x: 1 if x > 0 else 0
)
kilands_no_out1['purch'] = kilands_no_out1['number of purchases'].apply(
    lambda x: 1 if x > 0 else 0
)

In [26]:
kilands_no_out1.shape

(14805, 28)

In [27]:
# Renaming number of clicks column
    # Not in R code, just for personal use
kilands_no_out1 = kilands_no_out1.rename(columns={'number of clicks': 'number_of_clicks'})

# Renaming session length column
    # Not in R code, just for personal use
kilands_no_out1 = kilands_no_out1.rename(columns={'search session length (seconds)': 'search_length_seconds'})

### Hypothesis Testing ###

In [28]:
# * Hypotheses a: carts (Model 2) and b: purchase (Model 1) are now modeled as `Multilevel Logistic Regression`, i.e., logistic regression with random intercept per participant.
# * Hypothesis c: Products viewed (Model 3) is now modeled as `Multilevel Negative Binomial Regression`, i.e, negative binomial regression (count-model) with random intercept per participant.
# * Hypothesis d: Session Length (Model 4) is now modeled as `Multilevel Linear Regression`, i.e, just a regular linear mixed model with random intercept per participant.

#### Hypothesis a: carts ####

# Fit the model using pymer4
model_a_kil = Lmer(
    formula="carts ~ segment + (1|user)",
    data=kilands_no_out1,
    family="binomial"
)

# Fit the model 
result_a_kil = model_a_kil.fit(optimizer="bobyqa", maxfun=200000)

# Print the summary
print(result_a_kil)

# Same Intercept = -9.629
# Same standard errors = 0.272 in python, 0.2726052 in R
# Same random intercept variance = 126.077 with standard error = 11.228
# Same pvalue = 0.998 (very high..)

/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "purchase positions". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "purchase_position". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: Fu

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: carts~segment+(1|user)

Family: binomial	 Inference: parametric

Number of observations: 14805	 Groups: {'user': 7928.0}

Log-likelihood: -2054.747 	 AIC: 4115.494

Random effects:

             Name      Var     Std
user  (Intercept)  126.077  11.228

No random effect correlations specified

Fixed effects:

                                Estimate  2.5_ci  97.5_ci     SE     OR  \
(Intercept)                       -9.629 -10.162   -9.096  0.272  0.000   
segment202004_kilands_shuffle7     0.001  -0.554    0.556  0.283  1.001   

                                OR_2.5_ci  OR_97.5_ci  Prob  Prob_2.5_ci  \
(Intercept)                         0.000       0.000   0.0        0.000   
segment202004_kilands_shuffle7      0.575       1.743   0.5        0.365   

                                Prob_97.5_ci  Z-stat  P-val  Sig  
(Intercept)                            0.000 -35.397  0.000  ***  
segment202004_kilands_shuffle7    

In [29]:
# Regression table for Kilands hypothesis A comparable to broom.mixed::tidy(lmer_a, conf.int = T)
regression_table = result_a_kil[['Estimate', 'SE', 'OR', 'OR_2.5_ci', 'OR_97.5_ci', 'P-val']]

regression_table

,Estimate,SE,OR,OR_2.5_ci,OR_97.5_ci,P-val
(Intercept),-9.629,0.272,0.000,0.000,0.000,0.000
segment202004_kilands_shuffle7,0.001,0.283,1.001,0.575,1.743,0.998


In [30]:
kilands_no_out1['carts'].unique()

array([0, 1])

In [31]:
#### Hypothesis b: purchase ####

# Create a binary variable for 'num_segment'
kilands_no_out1['num_segment'] = kilands_no_out1['segment'].apply(lambda x: 1 if x == "HA" else 0)

# Fit the model using pymer4
model_b_kil = Lmer(
    formula="purch ~ segment + (1|user)",
    data=kilands_no_out1,
    family="binomial"
)

# Fit the model
result_b_kil = model_b_kil.fit(optimizer="bobyqa", maxfun=200000)

# Print the summary
print(result_b_kil)

/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "purchase positions". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "purchase_position". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '


[1] "Model failed to converge with max|grad| = 0.0537602 (tol = 0.002, component 1)"
[2] " \n"                                                                           

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         



/Users/adeline/anaconda3/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: purch~segment+(1|user)

Family: binomial	 Inference: parametric

Number of observations: 14805	 Groups: {'user': 7928.0}

Log-likelihood: -851.071 	 AIC: 1708.141

Random effects:

             Name      Var     Std
user  (Intercept)  185.971  13.637

No random effect correlations specified

Fixed effects:

                                Estimate  2.5_ci  97.5_ci     SE     OR  \
(Intercept)                      -11.207 -11.209  -11.204  0.001  0.000   
segment202004_kilands_shuffle7     0.013  -0.638    0.664  0.332  1.013   

                                OR_2.5_ci  OR_97.5_ci   Prob  Prob_2.5_ci  \
(Intercept)                         0.000       0.000  0.000        0.000   
segment202004_kilands_shuffle7      0.528       1.943  0.503        0.346   

                                Prob_97.5_ci    Z-stat  P-val  Sig  
(Intercept)                             0.00 -8518.469  0.000  ***  
segment202004_kilands_shuffl

In [32]:
# Display the regression table
print(result_b_kil[['Estimate', 'SE', 'OR', 'OR_2.5_ci', 'OR_97.5_ci', 'P-val']])

# Same Intercept = -11.207
# Same standard errors = 0.332 in python, 0.333 in R
# Similar random intercept variance = 185.971 with se = 13.637 in Python, 187.5 with 13.69 se in R
# Similar pvalue = 0.969 in python, 0.974 in R

                                Estimate     SE     OR  OR_2.5_ci  OR_97.5_ci  \
(Intercept)                      -11.207  0.001  0.000      0.000       0.000   
segment202004_kilands_shuffle7     0.013  0.332  1.013      0.528       1.943   

                                P-val  
(Intercept)                     0.000  
segment202004_kilands_shuffle7  0.969  


In [33]:
### Hypothesis c: clicks ###

# Fit the model
model_c_kil = smf.glm(
    formula="number_of_clicks ~ segment", 
    data=kilands_no_out1, 
    family=sm.families.NegativeBinomial()
).fit()

# Print summary
model_c_kil.summary()

# pymer4 doesn't support negative binomial regression, so I had to use smf.glm with statsmodels. 
# This function doesn't use random effects, which is the reason why the results are very different 

# Python intercept: -0.0145, R intercept: -0.35369
# Python standard error: 0.017, R standard error: 0.02336
# Diff. pvalue = 0.753 in python, 0.974 in R


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       number_of_clicks   No. Observations:                14805
Model:                            GLM   Df Residuals:                    14803
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -20338.
Date:                Thu, 28 Nov 2024   Deviance:                       15195.
Time:                        17:07:08   Pearson chi2:                 2.37e+04
No. Iterations:                     5   Pseudo R-squ. (CS):          6.706e-06
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -0.0145      0.017     -0.872      0.383      -0.047       0.018
segment[T.202004_kilands_shuffle7]    -0.0074      0.023     -0.315      0.753      -0.053       0.038
======================================================================================================
"""

In [34]:
### Hypothesis d: session length ###

model_d = Lmer(
    formula="search_length_seconds ~ segment + (1|user)",
    data=kilands_no_out1
)

# Fit the model
result_d = model_d.fit()

result_d

# Python intercept = 2.061, R intercept = 2.061 (same)
# Python standard error = 4.348, R standard error = 4.348
# Python random intercept variance = 6368.966 with se = 79.806 in Python, R riv = 6369 with 79.81
# P-value = 0.635 in python, 0.974 in R

/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "purchase positions". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "purchase_position". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: Fu

Linear mixed model fit by REML [’lmerMod’]
Formula: search_length_seconds~segment+(1|user)

Family: gaussian	 Inference: parametric

Number of observations: 14805	 Groups: {'user': 7928.0}

Log-likelihood: -101863.212 	 AIC: 203734.424

Random effects:

                 Name        Var      Std
user      (Intercept)   6368.966   79.806
Residual               49942.263  223.478

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),101.641,95.595,107.688,3.085,6877.540,32.949,0.000,***
segment202004_kilands_shuffle7,2.061,-6.461,10.583,4.348,6795.529,0.474,0.635,


## Cyberfoto Data: High Attractiveness (HA)

### Data Wrangling and Cleaning

In [35]:
cyberfoto_df.head()

,search id,user,session,segment,query,slot,platform,search start,search end time,number of add-to-carts,...,number of purchases,number of searches in current session,search session length (seconds),number of products displayed to user,number of sessions,whole session length,time to add-to-cart (seconds),time to first click (seconds),time to purchase,toggled filters
0,u:008G2FLphtEuSvuq/s:9TZB86uWh5JatbGL/q:q=x-t3...,008G2FLphtEuSvuq,9TZB86uWh5JatbGL,202004_gandalf_rel,X-t30,NaN,mobile,2020-04-20T18:38:47.611,2020-04-20T18:38:47.611,0,...,0,1,0,19,2,0,NaN,NaN,NaN,NaN
1,u:008G2FLphtEuSvuq/s:PsVPZAiJpPUZCTJx/q:q=t30/...,008G2FLphtEuSvuq,PsVPZAiJpPUZCTJx,202004_gandalf_rel,T30,NaN,mobile,2020-04-20T23:38:51.378,2020-04-20T23:38:51.378,0,...,0,1,17,24,2,17,NaN,17.0,NaN,NaN
2,u:00RhFdGXHGSDE1Wt/s:BxLPI4vKABZDOrnM/q:q=airp...,00RhFdGXHGSDE1Wt,BxLPI4vKABZDOrnM,202004_kilands_shuffle7,airpods pro,NaN,mobile,2020-04-24T20:34:14.007,2020-04-24T20:34:14.007,0,...,0,1,4,1,2,4,NaN,4.0,NaN,NaN
3,u:00RhFdGXHGSDE1Wt/s:xDm9Cldi5S1kHvTR/q:q=airp...,00RhFdGXHGSDE1Wt,xDm9Cldi5S1kHvTR,202004_kilands_shuffle7,Airpods,NaN,mobile,2020-04-24T21:02:08.957,2020-04-24T21:02:08.957,0,...,0,1,31,9,2,31,NaN,27.0,NaN,NaN
4,u:00btJth2TCL5RQjK/s:sW7TCyrnxTIaOfrh/q:q=kamo...,00btJth2TCL5RQjK,sW7TCyrnxTIaOfrh,202004_kilands_shuffle7,objektivskydd kamoflage,NaN,mobile,2020-05-05T06:02:25.128,2020-05-05T06:02:25.128,0,...,0,1,0,14,1,0,NaN,NaN,NaN,NaN


In [60]:
cyberfoto_df.value_counts()

Series([], Name: count, dtype: int64)

In [36]:
cyberfoto_df.shape

(37005, 24)

In [61]:
cyberfoto_df.columns

Index(['search id', 'user', 'session', 'segment', 'query', 'slot', 'platform',
       'search start', 'search end time', 'number of add-to-carts',
       'number of clicks', 'first click rank', 'first purchase rank',
       'order value', 'number of purchases',
       'number of searches in current session',
       'search session length (seconds)',
       'number of products displayed to user', 'number of sessions',
       'whole session length', 'time to add-to-cart (seconds)',
       'time to first click (seconds)', 'time to purchase', 'toggled filters'],
      dtype='object')

In [62]:
kilands_df.columns

Index(['search id', 'user', 'session', 'segment', 'query', 'slot', 'platform',
       'search start', 'search end time', 'number of add-to-carts',
       'number of clicks', 'first click rank', 'first purchase rank',
       'order value', 'number of purchases',
       'number of searches in current session',
       'search session length (seconds)',
       'number of products displayed to user', 'number of sessions',
       'whole session length', 'time to add-to-cart (seconds)',
       'time to first click (seconds)', 'time to purchase', 'toggled filters',
       'click positions', 'purchase positions'],
      dtype='object')

In [42]:
### Removing Outliers ###

# Define bounds for exclusion: search session length
upper_bound_length_cyber = cyberfoto_df['search session length (seconds)'].mean() + \
                             2.5 * cyberfoto_df['search session length (seconds)'].std()
print(upper_bound_length_cyber)

# Count the number of observations with search session length exceeding the upper bound
num_outliers_search = len(cyberfoto_df[cyberfoto_df['search session length (seconds)'] > upper_bound_length_cyber])
print(num_outliers_search)

# Creating new dataset with search session outliers removed
cyber_no_out = cyberfoto_df[cyberfoto_df['search session length (seconds)'] <= upper_bound_length_cyber]

## Same bounds for exclusion and number of outliers in Python and R code

418.98503037554354
949


In [43]:
# Define bounds for exclusion: clicks
upper_bound_click_cyber = cyberfoto_df['number of clicks'].mean() + \
                             2.5 * cyberfoto_df['number of clicks'].std()
print(upper_bound_click_cyber)

# Count the number of observations with clicks exceeding the upper bound
num_outliers_search = len(cyber_no_out[cyber_no_out['number of clicks'] > upper_bound_click_cyber])
print(num_outliers_search)

3.039139238364073
406


In [46]:
# Creating new dataset with search session outliers removed
cyber_no_out1 = cyber_no_out[cyber_no_out['number of clicks'] <= upper_bound_click_cyber]

In [49]:
# Recode High Attractiveness (HA) and Low Attractiveness (LA)
cyber_no_out1.loc[cyber_no_out1['segment'] == "202004_gandalf_rel", 'segment'] = "HA"
cyber_no_out1.loc[cyber_no_out1['segment'] == "202004_kilands_shuffle7", 'segment'] = "LA"

# Keep only certain cols: search ID (1), user (2), segment (4), platform (7), 15, 10, 11, 12, 13,
##, 14, 17, 20, 21, 22, 23, 25, 26, 18
columns_to_keep = [
    cyber_no_out.columns[i - 1] for i in 
    [1, 2, 4, 7, 15, 11, 17, 12, 13, 14, 20, 21, 23, 22, 18]
]
cyber_no_out1 = cyber_no_out[columns_to_keep]

In [74]:
# Recode attention_click
cyber_no_out1['attention_click'] = cyber_no_out1['first click rank'].apply(
    lambda x: "Top" if x <= 6 else ("Bottom" if x > 6 else x))

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/3817770656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyber_no_out1['attention_click'] = cyber_no_out1['first click rank'].apply(


In [75]:
# Recode attention_purchase
cyber_no_out1['attention_purchase'] = cyber_no_out1['first purchase rank'].apply(
    lambda x: "Top" if x <= 6 else ("Bottom" if x > 6 else x))

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/835098656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyber_no_out1['attention_purchase'] = cyber_no_out1['first purchase rank'].apply(


In [57]:
cyber_no_out1.head()

,search id,user,segment,platform,number of purchases,number of clicks,search session length (seconds),first click rank,first purchase rank,order value,whole session length,time to add-to-cart (seconds),time to purchase,time to first click (seconds),number of products displayed to user,attention_click,attention_purchase
0,u:008G2FLphtEuSvuq/s:9TZB86uWh5JatbGL/q:q=x-t3...,008G2FLphtEuSvuq,202004_gandalf_rel,mobile,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,19,NaN,NaN
1,u:008G2FLphtEuSvuq/s:PsVPZAiJpPUZCTJx/q:q=t30/...,008G2FLphtEuSvuq,202004_gandalf_rel,mobile,0,1,17,1.0,NaN,NaN,17,NaN,NaN,17.0,24,Top,NaN
2,u:00RhFdGXHGSDE1Wt/s:BxLPI4vKABZDOrnM/q:q=airp...,00RhFdGXHGSDE1Wt,202004_kilands_shuffle7,mobile,0,1,4,0.0,NaN,NaN,4,NaN,NaN,4.0,1,Top,NaN
3,u:00RhFdGXHGSDE1Wt/s:xDm9Cldi5S1kHvTR/q:q=airp...,00RhFdGXHGSDE1Wt,202004_kilands_shuffle7,mobile,0,2,31,4.0,NaN,NaN,31,NaN,NaN,27.0,9,Top,NaN
4,u:00btJth2TCL5RQjK/s:sW7TCyrnxTIaOfrh/q:q=kamo...,00btJth2TCL5RQjK,202004_kilands_shuffle7,mobile,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,14,NaN,NaN


In [63]:
# Replace semicolons in 'click positions' and 'purchase positions' with ' '
#cyber_no_out1['clicks_position'] = cyber_no_out1['click positions'].str.replace(";", " ")
#cyber_no_out1['purchase_position'] = cyber_no_out1['purchase positions'].str.replace(";", " ")

In [65]:
# Recode top_clicks and bottom_clicks
# cyber_no_out1['top_clicks'] = cyber_no_out1['clicks_position'].apply(
#     lambda x: sum(int(pos) <= 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
# )
# cyber_no_out1['bottom_clicks'] = cyber_no_out1['clicks_position'].apply(
#     lambda x: sum(int(pos) > 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
# )

In [ ]:
# Recode top_purch and bottom_purch
# cyber_no_out1['top_purch'] = cyber_no_out1['purchase_position'].apply(
#     lambda x: sum(int(pos) <= 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
# )

# cyber_no_out1['bottom_purch'] = cyber_no_out1['purchase_position'].apply(
#     lambda x: sum(int(pos) > 6 for pos in x.split() if pos.isdigit()) if pd.notna(x) else 0
# )

In [66]:
# Create dummy binary variables purch that captures whether an add-to-cart/purchase has happened
cyber_no_out1['purch'] = cyber_no_out1['number of purchases'].apply(
    lambda x: 1 if x > 0 else 0
)

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/2549143029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyber_no_out1['purch'] = cyber_no_out1['number of purchases'].apply(


In [82]:
# Renaming number of clicks column
    # Not in R code, just for personal use
cyber_no_out1 = cyber_no_out1.rename(columns={'number of clicks': 'number_of_clicks'})

# Renaming session length column
    # Not in R code, just for personal use
cyber_no_out1 = cyber_no_out1.rename(columns={'search session length (seconds)': 'search_length_seconds'})

In [79]:
#### HYPOTHESIS TESTING ####

# * Hypotheses b: purchase (Model 1) are now modeled as `Multilevel Logistic Regression`, i.e., logistic regression with random intercept per participant.
# * Hypothesis c: Products viewed (Model 3) is now modeled as `Multilevel Negative Binomial Regression`, i.e, negative binomial regression (count-model) with random intercept per participant.
# * Hypothesis d: Session Length (Model 4) is now modeled as `Multilevel Linear Regression`, i.e, just a regular linear mixed model with random intercept per participant.

#### Hypothesis b: purchase ####

# Create a binary variable for 'num_segment'
cyber_no_out1['num_segment'] = cyber_no_out1['segment'].apply(lambda x: 1 if x == "HA" else 0)

# Fit the model using pymer4
model_b_cyb = Lmer(
    formula="purch ~ segment + (1|user)",
    data=cyber_no_out1,
    family="binomial"
)

# Fit the model
result_b_cyb = model_b_cyb.fit(optimizer="bobyqa", maxfun=200000)

# Print the summary
print(result_b_cyb)

# Python intercept = 0.165, R intercept = 0.165 (same)
# Python standard error = 0.553, R standard error = 0.553 (same)
# Python random intercept variance = 197.894 with se = 14.067 in Python, R riv = 197.9 with 14.07 se
# P-value = 0.766 in python, 0.766 in R

/var/folders/jf/tn4cnxdn127dtc8x_7vb71000000gn/T/ipykernel_38280/951184887.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyber_no_out1['num_segment'] = cyber_no_out1['segment'].apply(lambda x: 1 if x == "HA" else 0)
/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "attention_click". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: purch~segment+(1|user)

Family: binomial	 Inference: parametric

Number of observations: 36056	 Groups: {'user': 15237.0}

Log-likelihood: -686.634 	 AIC: 1379.268

Random effects:

             Name      Var     Std
user  (Intercept)  197.894  14.067

No random effect correlations specified

Fixed effects:

                                Estimate  2.5_ci  97.5_ci     SE     OR  \
(Intercept)                      -12.620  -13.59  -11.650  0.495  0.000   
segment202004_kilands_shuffle7     0.165   -0.92    1.249  0.553  1.179   

                                OR_2.5_ci  OR_97.5_ci   Prob  Prob_2.5_ci  \
(Intercept)                         0.000       0.000  0.000        0.000   
segment202004_kilands_shuffle7      0.399       3.487  0.541        0.285   

                                Prob_97.5_ci  Z-stat  P-val  Sig  
(Intercept)                            0.000 -25.500  0.000  ***  
segment202004_kilands_shuffle7 

In [81]:
cyber_no_out1

,search id,user,segment,platform,number of purchases,number of clicks,search session length (seconds),first click rank,first purchase rank,order value,whole session length,time to add-to-cart (seconds),time to purchase,time to first click (seconds),number of products displayed to user,attention_click,attention_purchase,purch,num_segment
0,u:008G2FLphtEuSvuq/s:9TZB86uWh5JatbGL/q:q=x-t3...,008G2FLphtEuSvuq,202004_gandalf_rel,mobile,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,19,NaN,NaN,0,0
1,u:008G2FLphtEuSvuq/s:PsVPZAiJpPUZCTJx/q:q=t30/...,008G2FLphtEuSvuq,202004_gandalf_rel,mobile,0,1,17,1.0,NaN,NaN,17,NaN,NaN,17.0,24,Top,NaN,0,0
2,u:00RhFdGXHGSDE1Wt/s:BxLPI4vKABZDOrnM/q:q=airp...,00RhFdGXHGSDE1Wt,202004_kilands_shuffle7,mobile,0,1,4,0.0,NaN,NaN,4,NaN,NaN,4.0,1,Top,NaN,0,0
3,u:00RhFdGXHGSDE1Wt/s:xDm9Cldi5S1kHvTR/q:q=airp...,00RhFdGXHGSDE1Wt,202004_kilands_shuffle7,mobile,0,2,31,4.0,NaN,NaN,31,NaN,NaN,27.0,9,Top,NaN,0,0
4,u:00btJth2TCL5RQjK/s:sW7TCyrnxTIaOfrh/q:q=kamo...,00btJth2TCL5RQjK,202004_kilands_shuffle7,mobile,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,14,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37000,u:zzaKyVjD9LmtbO45/s:AHxsLHtjFjWeLHT1/q:q=21mm...,zzaKyVjD9LmtbO45,202004_gandalf_rel,mobile,0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,24,NaN,NaN,0,0
37001,u:zzcQSxWDq8qX3j5Z/s:1OnqipD7sVTigIIE/q:q=15-3...,zzcQSxWDq8qX3j5Z,202004_gandalf_rel,mobile,0,1,8,5.0,NaN,NaN,8,NaN,NaN,8.0,6,Top,NaN,0,0
37002,u:zzcQSxWDq8qX3j5Z/s:8R8BeyM3yp02zl1M/q:q=7d i...,zzcQSxWDq8qX3j5Z,202004_gandalf_rel,mobile,0,0,0,NaN,NaN,NaN,414,NaN,NaN,NaN,10,NaN,NaN,0,0
37003,u:zzcQSxWDq8qX3j5Z/s:8R8BeyM3yp02zl1M/q:q=a630...,zzcQSxWDq8qX3j5Z,202004_gandalf_rel,mobile,0,0,0,NaN,NaN,NaN,414,NaN,NaN,NaN,17,NaN,NaN,0,0


In [83]:
### Hypothesis c: clicks ###

# Fit the model
model_c_cyb = smf.glm(
    formula="number_of_clicks ~ segment", 
    data=cyber_no_out1, 
    family=sm.families.NegativeBinomial()
).fit()

# Print summary
model_c_cyb.summary()

# pymer4 doesn't support negative binomial regression, so I had to use smf.glm with statsmodels. 
# This function doesn't use random effects, which is the reason why the results are very different 

# Python intercept: -0.6269, R intercept: -0.35369
# Python standard error: 0.013, R standard error: 0.02336
# Similar pvalue = 0.289 in python, 0.454 in R

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       number_of_clicks   No. Observations:                36056
Model:                            GLM   Df Residuals:                    36054
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -35943.
Date:                Mon, 02 Dec 2024   Deviance:                       27271.
Time:                        00:51:41   Pearson chi2:                 3.09e+04
No. Iterations:                     5   Pseudo R-squ. (CS):          3.121e-05
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -0.6269      0.013    -49.997      0.000      -0.652      -0.602
segment[T.202004_kilands_shuffle7]     0.0189      0.018      1.061      0.289      -0.016       0.054
======================================================================================================
"""

In [84]:
### Hypothesis d: session length ###

model_d = Lmer(
    formula="search_length_seconds ~ segment + (1|user)",
    data=cyber_no_out1
)

# Fit the model
result_d = model_d.fit()

result_d

# Python intercept = -0.463, R intercept = -0.4632 (same)
# Python standard error = 0.627, R standard error = 0.627
# Python random intercept variance = 125.845 with se = 11.218 in Python, R riv = 125.8 with 11.22
# P-value = 0.46 in python, 0.46 in R


/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "attention_click". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "attention_purchase". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/adeline/anaconda3/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: Futu

Linear mixed model fit by REML [’lmerMod’]
Formula: search_length_seconds~segment+(1|user)

Family: gaussian	 Inference: parametric

Number of observations: 36056	 Groups: {'user': 15237.0}

Log-likelihood: -195806.674 	 AIC: 391621.348

Random effects:

                 Name       Var     Std
user      (Intercept)   125.845  11.218
Residual               2935.729  54.182

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),22.476,21.615,23.338,0.439,7557.819,51.149,0.00,***
segment202004_kilands_shuffle7,-0.463,-1.692,0.766,0.627,7425.014,-0.739,0.46,
